<a href="https://colab.research.google.com/github/veritaem/hackathon-day1/blob/master/lyric_book.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
!pip install NLTK

In [2]:
!pip install lyricsgenius

    100% |████████████████████████████████| 61kB 3.3MB/s 
    100% |████████████████████████████████| 92kB 4.7MB/s 
spacy 2.0.18 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
google-colab 0.0.1a1 has requirement requests~=2.18.0, but you'll have requests 2.21.0 which is incompatible.
cufflinks 0.14.6 has requirement plotly>=3.0.0, but you'll have plotly 1.12.12 which is incompatible.
  Found existing installation: requests 2.18.4
    Uninstalling requests-2.18.4:
      Successfully uninstalled requests-2.18.4
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  [requests]
You must restart the runtime in order to use newly installed versions.


In [3]:
!pip install pronouncing

    100% |████████████████████████████████| 942kB 7.2MB/s 
  Running setup.py bdist_wheel for pronouncing ... - done
  Stored in directory: /root/.cache/pip/wheels/81/fd/e8/fb1a226f707c7e20dbed4c43f81b819d279ffd3b0e2f06ee13
Successfully built pronouncing


In [4]:
import pandas as pd
import numpy as np
import requests
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML, display
from bs4 import BeautifulSoup
from nltk.tokenize import RegexpTokenizer
import lyricsgenius as genius
import sys
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from nltk.stem import PorterStemmer
from datetime import datetime
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import spacy
from collections import Counter
from os import path
from PIL import Image
from keras.models import model_from_json
import pickle
pd.options.mode.chained_assignment = None
display(HTML("<style>.container { width:100% !important; }</style>"))
%config InlineBackend.figure_format = 'png'

Using TensorFlow backend.


In [0]:
def collect_songs_from_billboard(start_year,end_year):    
    years = np.arange(start_year, end_year + 1).astype(int)
    dataset = pd.DataFrame()
    url_list = []
    all_years = pd.DataFrame()
    alternate_site_collection_range = np.arange(start_year, 2017)

    for i in range (0, len(years)):
        url_list.append("https://www.billboard.com/charts/year-end/" + str(years[i]) + "/hot-100-songs")      
    for i in range(0, len(url_list)):
        if years[i] in alternate_site_collection_range:
            sys.stdout.write("\r" + "Collecting Songs from " +str(years[i]) + " via http://www.bobborst.com")
            sys.stdout.flush()    
            url = "http://www.bobborst.com/popculture/top-100-songs-of-the-year/?year=" + str(years[i])
            page = requests.get(url)
            soup = BeautifulSoup(page.content, "html.parser")
            table = soup.find('table', {'class': 'sortable alternate songtable'})
            rows = table.find_all('tr')
            for j in range(2,102):
                columns = rows[j].find_all('td')
                row = {
                    "Rank": columns[0].get_text(strip=True),
                    "Artist": columns[1].get_text(strip=True),
                    "Song Title": columns[2].get_text(strip=True),
                    "Year": years[i]
                }
                dataset = dataset.append(row, ignore_index=True)
            
        else:
            sys.stdout.write("\r" + "Collecting Songs from " +str(years[i]) + " via https://www.billboard.com")
            sys.stdout.flush()
            url = "https://www.billboard.com/charts/year-end/" + str(years[i]) + "/hot-100-songs"
            page = requests.get(url)
            soup = BeautifulSoup(page.content, "html.parser")
            all_ranks = soup.find_all("div", class_="ye-chart-item__rank")
            all_titles = soup.find_all('div', class_="ye-chart-item__title")
            all_artists = soup.find_all("div", class_="ye-chart-item__artist")
            for j in range (0, len(all_ranks)):
                row = {
                    "Rank": all_ranks[j].get_text(strip=True),
                    "Song Title": all_titles[j].get_text(strip=True),
                    "Artist": all_artists[j].get_text(strip=True),
                    "Year": years[i]
                }
                dataset = dataset.append(row, ignore_index=True)
    dataset['Year'] = dataset['Year'].astype(int)
    return dataset
    

In [0]:
def plot_year_counts(dataset, X, y, title):
    '''Function to plot the year count summaries.
    
    Parameters:

    dataset (dataframe): pandas dataframe
    X (string): the x axis column
    y (string): the y axis column
    title (string): the title of the chart
    
    Outputs a matplotlib lineplot
    '''
    characteristics = dataset.groupby(X).count()
    mpl.rcParams['figure.figsize'] = (35,10,)
    #all_songs.groupby('Year').count().plot(kind='bar')
    sns.barplot(y=characteristics[y], x=characteristics.index)
    plt.title(title)
    plt.ylabel("Number of Songs")
    plt.xticks(rotation=90)


In [0]:
def add_spacy_data(dataset, feature_column):
    '''
    Grabs the verb, adverb, noun, and stop word Parts of Speech (POS) 
    tokens and pushes them into a new dataset. returns an 
    enriched dataset.
    
    Parameters:
    
    dataset (dataframe): the dataframe to parse
    feature_column (string): the column to parse in the dataset.
    
    Returns: 
    dataframe
    '''
    
    verbs = []
    nouns = []
    adverbs = []
    corpus = []
    nlp = spacy.load('en_core_web_sm')
    ##
    for i in range (0, len(dataset)):
        print("Extracting verbs and topics from record {} of {}".format(i+1, len(dataset)), end = "\r")
        song = dataset.iloc[i][feature_column]
        doc = nlp(song)
        spacy_dataframe = pd.DataFrame()
        for token in doc:
            if token.lemma_ == "-PRON-":
                    lemma = token.text
            else:
                lemma = token.lemma_
            row = {
                "Word": token.text,
                "Lemma": lemma,
                "PoS": token.pos_,
                "Stop Word": token.is_stop
            }
            spacy_dataframe = spacy_dataframe.append(row, ignore_index = True)
        verbs.append(" ".join(spacy_dataframe["Lemma"][spacy_dataframe["PoS"] == "VERB"].values))
        nouns.append(" ".join(spacy_dataframe["Lemma"][spacy_dataframe["PoS"] == "NOUN"].values))
        adverbs.append(" ".join(spacy_dataframe["Lemma"][spacy_dataframe["PoS"] == "ADV"].values))
        corpus_clean = " ".join(spacy_dataframe["Lemma"][spacy_dataframe["Stop Word"] == False].values)
        corpus_clean = re.sub(r'[^A-Za-z0-9]+', ' ', corpus_clean)   
        corpus.append(corpus_clean)
    dataset['Verbs'] = verbs
    dataset['Nouns'] = nouns
    dataset['Adverbs'] = adverbs
    dataset['Corpus'] = corpus
    return dataset

In [0]:
def prep_corpus(raw_string):
    verbs = []
    nouns = []
    adverbs = []
    corpus = []
    nlp = spacy.load('en_core_web_sm')

    doc = nlp(raw_string)
    spacy_dataframe = pd.DataFrame()
    for token in doc:
        if token.lemma_ == "-PRON-":
                lemma = token.text
        else:
            lemma = token.lemma_
        row = {
            "Word": token.text,
            "Lemma": lemma,
            "PoS": token.pos_,
            "Stop Word": token.is_stop
        }
        spacy_dataframe = spacy_dataframe.append(row, ignore_index = True)
    verbs.append(" ".join(spacy_dataframe["Lemma"][spacy_dataframe["PoS"] == "VERB"].values))
    nouns.append(" ".join(spacy_dataframe["Lemma"][spacy_dataframe["PoS"] == "NOUN"].values))
    adverbs.append(" ".join(spacy_dataframe["Lemma"][spacy_dataframe["PoS"] == "ADV"].values))
    corpus_clean = " ".join(spacy_dataframe["Lemma"][spacy_dataframe["Stop Word"] == False].values)
    corpus_clean = re.sub(r'[^A-Za-z0-9]+', ' ', corpus_clean)   

    return corpus_clean

In [0]:
def wordcloud(text, max_words):
    '''
    Wrapper around Wordcloud that increases quality, picks a specific font,
    and puts it on a white background
    '''
    
    wordcloud = WordCloud(font_path='C:\WINDOWS\FONTS\PERTILI.TTF',
                          width = 4000,
                          height = 3000,
                          background_color="white",
                          max_words = max_words                          
                         ).generate(text)
    plt.figure(figsize=(40,25))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad=0)
    plt.show()
    return;

In [0]:
def map_popular_terms(dataset, feature_column, year_column):
    '''Function that counts the frequency of occurences of words in a dataset
    column. Returns a new dataset with those frequencies'''
    frequencies = pd.DataFrame()
    years = dataset[year_column].unique().tolist()
    for i in range (0, len(years)):
        year_corpus = str(dataset[feature_column][dataset[year_column] == years[i]].tolist())
        tokens = year_corpus.split(" ")
        counts = Counter(tokens)
        frequencies = frequencies.append({
            "Year": years[i],
            "Most Common Terms": counts.most_common(n=100)
        }, ignore_index=True)
    frequencies['Year'] = frequencies['Year'].astype(int)
    return frequencies
    

In [0]:
def map_common_words(dataset):
    '''Maps common words from across multiple columns in a dataset to 
    identify terms that show up in all columns. Normally used with the 
    outputs of map_popular_terms. returns the common words'''
    common_words = []
    for words in dataset['Most Common Terms'][0]:
        common_words.append(words[0])

    for i in range (0, len(dataset)):
        check_list = []
        year_list = dataset['Most Common Terms'][i]
        for words in year_list:
            check_list.append(words[0])
        common_words = [x for x in common_words if x  in check_list]
    return common_words

In [0]:
def get_common_frequency(term_list, frequency_list):
    '''Finds the frequency of occurence of terms in a list and then
    returns them in a new dataframe organized by year'''
    common_word_frequency_per_year = pd.DataFrame()
    for i in range(0, len(term_list)):
        word_frequency = []
        for j in range(0, len(frequency_list)):
            current_year = frequency_list['Year'][j]
            current_year_terms = frequency_list['Most Common Terms'][j]
            for words in current_year_terms:
                    if term_list[i] in words[0]:
                        word_frequency.append(words[1])
                        #print(words[1])
                        break
        current_word = term_list[i]
        common_word_frequency_per_year[str(current_word)] = word_frequency
    common_word_frequency_per_year["Year"] = np.arange(1970,2019)
    common_word_frequency_per_year = common_word_frequency_per_year.set_index("Year")
    return common_word_frequency_per_year

In [13]:
all_songs = collect_songs_from_billboard(1970, 2018)

In [14]:
all_songs.shape

(4900, 4)

In [0]:
api = genius.Genius("dKPwHh2dMS-ZULc5KY4KAXtbtDgR0yYsbLtNxuzXcEoFHor9agY8kKbFeqblZwoe",sleep_time=0.01, verbose=False)

In [25]:
all_song_data = pd.DataFrame()
start_time = datetime.now()
print("Started at {}".format(start_time))
for i in range(0, len(all_songs)):
    rolling_pct = int((i/len(all_songs))*100)
    print(str(rolling_pct) + "% complete." + " Collecting Record " + str(i) +" of " +
          str(len(all_songs)) +". Year " + str(all_songs.iloc[i]['Year']) + "." + " Currently collecting " + 
          all_songs.iloc[i]['Song Title'] + " by " + all_songs.iloc[i]['Artist'] + " "*50, end="\r")
    song_title = all_songs.iloc[i]['Song Title']
    song_title = re.sub(" and ", " & ", song_title)
    artist_name = all_songs.iloc[i]['Artist']
    artist_name = re.sub(" and ", " & ", artist_name)

    try:
        song = api.search_song(song_title, artist=artist_name)
        #print(song)
        song_album = song.album
        song_album_url = song.album_url
        featured_artists = song.featured_artists
        song_lyrics = re.sub("\n", " ", song.lyrics) #Remove newline breaks, we won't need them.
        song_media = song.media
        song_url = song.url
        song_writer_artists = song.writer_artists
        song_year = song.year
    except:
        song_album = "null"
        song_album_url = "null"
        featured_artists = "null"
        song_lyrics = "null"
        song_media = "null"
        song_url = "null"
        song_writer_artists = "null"
        song_year = "null"
        
    row = {
        "Year": all_songs.iloc[i]['Year'],
        "Rank": all_songs.iloc[i]['Rank'],
        "Song Title": all_songs.iloc[i]['Song Title'],
        "Artist": all_songs.iloc[i]['Artist'],
        "Album": song_album,
        "Album URL": song_album_url,
        "Featured Artists": featured_artists,
        "Lyrics": song_lyrics,
        "Media": song_media,
        "Song URL": song_url,
        "Writers": song_writer_artists,
        "Release Date": song_year
    }
    all_song_data = all_song_data.append(row, ignore_index=True)
end_time = datetime.now()
print("\nCompleted at {}".format(start_time))
print("Total time to collect: {}".format(end_time - start_time))



Started at 2019-01-21 21:19:45.506199

Completed at 2019-01-21 21:19:45.506199
Total time to collect: 1:22:34.156970


In [28]:
all_song_data.head()

,Album,Album URL,Artist,Featured Artists,Lyrics,Media,Rank,Release Date,Song Title,Song URL,Writers,Year
0,Bridge Over Troubled Water,https://genius.com/albums/Simon-and-garfunkel/...,Simon and Garfunkel,[],[Verse 1] When you're weary Feeling small When...,[{'native_uri': 'spotify:track:6l8EbYRtQMgKOyc...,1,1970-01-26,Bridge Over Troubled Water,https://genius.com/Simon-and-garfunkel-bridge-...,"[{'api_path': '/artists/15474', 'header_image_...",1970.0
1,Close to You,https://genius.com/albums/Carpenters/Close-to-you,Carpenters,[],Why do birds suddenly appear Every time you ar...,[{'native_uri': 'spotify:track:79SE4x5ZROfEqaN...,2,1970-05-15,(They Long To Be) Close To You,https://genius.com/Carpenters-they-long-to-be-...,"[{'api_path': '/artists/412388', 'header_image...",1970.0
2,Bojack Horseman Season 1 Scripts,https://genius.com/albums/Bojack-horseman/Boja...,Guess Who,"[{'api_path': '/artists/262399', 'header_image...",INT. BAR BOJACK is sitting at the bar counter...,[],3,None,American Woman / No Sugar Tonight,https://genius.com/Bojack-horseman-bojack-hate...,"[{'api_path': '/artists/262399', 'header_image...",1970.0
3,Raindrops Keep Fallin' on My Head,https://genius.com/albums/Bj-thomas/Raindrops-...,B.J. Thomas,[],Raindrops keep fallin' on my head And just lik...,[{'native_uri': 'spotify:track:7BSIoadwe5hNIO1...,4,None,Raindrops Keep Fallin' On My Head,https://genius.com/Bj-thomas-raindrops-keep-fa...,"[{'api_path': '/artists/412388', 'header_image...",1970.0
4,War & Peace,https://genius.com/albums/Edwin-starr/War-peace,Edwin Starr,[],"[Intro] (War, what is it bad for?) Absolutely ...",[{'native_uri': 'spotify:track:7bfWzWm54GVlb8X...,5,1970-06-10,War,https://genius.com/Edwin-starr-war-lyrics,"[{'api_path': '/artists/326091', 'header_image...",1970.0


In [0]:
all_song_data.to_html('songdata.html')

In [0]:
all_song_data.to_csv("all_songs_data.csv")
all_song_data.to_json("all_song_data.json", orient='records')

In [35]:
songs_wl_df = all_song_data.dropna(subset=['Lyrics'])
songs_wl_df.shape

(4900, 12)

In [36]:
songs_wl_df['Lyrics'][0]

"[Verse 1] When you're weary Feeling small When tears are in your eyes I will dry them all I'm on your side When times get rough And friends just can't be found Like a bridge over troubled water I will lay me down Like a bridge over troubled water I will lay me down  [Verse 2] When you're down and out When you're on the street When evening falls so hard I will comfort you I'll take your part When darkness comes And pain is all around Like a bridge over troubled water I will lay me down Like a bridge over troubled water I will lay me down  [Verse 3] Sail on Silver Girl Sail on by Your time has come to shine All your dreams are on their way See how they shine If you need a friend I'm sailing right behind Like a bridge over troubled water I will ease your mind Like a bridge over troubled water I will ease your mind"

In [0]:
def strip_verse(X):
  str(songs_wl_df['Lyrics'][X]).strip('[Verse 1]')
  str(songs_wl_df['Lyrics'][X]).strip('[Verse 2]')
  str(songs_wl_df['Lyrics'][X]).strip('[Verse 3]')
  return songs_wl_df['Lyrics'][X]

In [51]:
strip_verse(0)

"[Verse 1] When you're weary Feeling small When tears are in your eyes I will dry them all I'm on your side When times get rough And friends just can't be found Like a bridge over troubled water I will lay me down Like a bridge over troubled water I will lay me down  [Verse 2] When you're down and out When you're on the street When evening falls so hard I will comfort you I'll take your part When darkness comes And pain is all around Like a bridge over troubled water I will lay me down Like a bridge over troubled water I will lay me down  [Verse 3] Sail on Silver Girl Sail on by Your time has come to shine All your dreams are on their way See how they shine If you need a friend I'm sailing right behind Like a bridge over troubled water I will ease your mind Like a bridge over troubled water I will ease your mind"

In [115]:
import re
test =re.findall('[A-Z][^A-Z]*', songs_wl_df['Lyrics'][130])
test

['M',
 'A',
 'S',
 'O',
 'N ',
 'C',
 'I',
 'T',
 'Y ',
 'To get there you follow ',
 'Highway 58, going northeast of the city, and it is a good highway and new. ',
 'Or was new, that day we went up it. ',
 "You look up the highway and it is straight for miles, coming at you, with the black line down the center coming at and at you, black and slick and tarry- shining against the white of the slab, and the heat dazzles up from the white slab so that only the black line is clear, coming at you with the whine of the tires, and if you don't quit staring at that line and don't take a few deep breaths and slap yourself hard on the back of the neck you'll hypnotize yourself and you'll come to just at the moment when the right font wheel hookers over into the black dirt shoulder off the slab, and you'll try to jerk her back on but you can't because the slab is high like a curb, and maybe you'll try to reach to turn off course. ",
 'Then a nigger chopping cotton a mile away, he\'ll look up and 

In [0]:
from nltk.tokenize import word_tokenize, wordpunct_tokenize, sent_tokenize
tokens = [wordpunct_tokenize(t) for t in test]
punct = set(['.', ',', '!', ':', ';'])
filtered = [ [w for w in sentence if w not in punct ] for sentence in tokens]
last = [ sentence[len(sentence) - 1] for sentence in filtered]

In [117]:
import nltk
nltk.download('cmudict')

[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


True

In [0]:
from nltk.corpus import cmudict

In [119]:
[[(w, pron) for (word, pron) in cmudict.entries() if word == w] for w in last]

KeyboardInterrupt: ignored

In [0]:
syllables = \
  [[(word, len(pron)) for (word, pron) in cmudict.entries() if word == w] \
    for w in last]